# Download the test video

In [ ]:
!pip install pytube moviepy

In [ ]:
import os
import pytube
from moviepy.editor import *

# Function to download a YouTube video
def download_video(url, output_path):
    yt = pytube.YouTube(url)
    stream = yt.streams.get_highest_resolution()
    stream.download(output_path=output_path)


In [ ]:
# YouTube video URL
youtube_url = 'https://youtube.com/shorts/G3t_SbT4_k8?si=F8obLNelFhvbQPZL'

# Output folder
output_folder = 'downloads'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Download the YouTube video to the output folder
download_video(youtube_url, output_folder)

# Find the downloaded video file in the output folder
video_files = [f for f in os.listdir(output_folder) if f.endswith('.mp4')]

if not video_files:
    print("No video files found in the output folder.")
else:
    video_file = os.path.join(output_folder, video_files[0])

    # Convert the video to MP3

    print("Downloaded and converted to MP3 successfully!")

    # Clean up - delete the video file

Downloaded and converted to MP3 successfully!


# Extract part of the video

In [ ]:
videos_folder = "ultralytics"
output_folder = "ultralytics-out"
!mkdir videos_folder output_folder
!mv /content/downloads/popup.mp4 videos_folder/

mkdir: cannot create directory ‘videos_folder’: File exists
mkdir: cannot create directory ‘output_folder’: File exists


In [ ]:
#extract 10 seconds
!ffmpeg -ss 00:00:00 -to 00:00:05 -i /content/videos_folder/popup.mp4 -c copy output.mp4


In [ ]:
from IPython.display import HTML
from base64 import b64encode

# Open the video file in binary mode
mp4 = open('/content/output_folder/workouts_0.mp4','rb').read()

# Encode the video in base64
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

# Display the video
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

Output hidden; open in https://colab.research.google.com to view.

# Configure Ultralytics

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

In [ ]:
import os
import math
from ultralytics import YOLO
from ultralytics.solutions import ai_gym
import cv2
import cv2
import os
model = YOLO("yolov8n-pose.pt")

In [28]:
def draw_dotted_line(image, pt1, pt2, color, thickness=1, gap=5):
    dist = ((pt1[0]-pt2[0])**2 + (pt1[1]-pt2[1])**2)**.5
    pts = []
    for i in np.arange(0, dist, gap):
        r = i / dist
        x = int((pt1[0]*(1-r) + pt2[0]*r) + .5)
        y = int((pt1[1]*(1-r) + pt2[1]*r) + .5)
        p = (x, y)
        pts.append(p)

    for p in pts:
        cv2.circle(image, p, thickness, color, -1)


def getAngle(a, b, c):
    ang = math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
    return ang + 360 if ang < 0 else ang




In [ ]:


# Videos processing
videos_folder = "videos_folder"
output_folder = "output_folder"
annotated_frame = results[0].plot()


if not os.path.exists(output_folder):
    os.mkdir(output_folder)
for ind, vid in enumerate(os.listdir(videos_folder)):


    cap = cv2.VideoCapture(os.path.join(videos_folder, vid))
    #assert cap.isOpened(), "Error reading video file"

    # Video Writer
    video_writer = cv2.VideoWriter(os.path.join(output_folder, "workouts_{}.mp4".format(ind)),
                                   cv2.VideoWriter_fourcc(*'mp4v'),
                                   int(cap.get(5)),
                                   (int(cap.get(3)), int(cap.get(4))))

    frame_count = 0
    while cap.isOpened():
        success, im0 = cap.read()
        if not success:
            print("Video processing has been successfully completed.")
            break

        frame_count += 1
        results = model.predict(im0, conf=0.25,verbose=False)
        for result in results:
            kpts = result.keypoints
            # kpts_conf returns a confidence keypoint for each of the keypoints
            kpts_conf = kpts.conf
            # Define the indices of the keypoints you want to connect
            dotted_line_indices = [[5, 7, 9], [6, 8, 10], [12, 14, 16], [11, 13,15]]
            colors = [(0, 255, 0), (0, 255, 255), (255, 0, 0), (255, 255, 0),(0,255,125)] # Colors for the dotted lines
            color_waist = (255,255,255)
            waist_indices =[11,12]
            if kpts.conf is not None:

              for i, index_triplet in enumerate(dotted_line_indices): # iterate over the index triplets for dotted lines
                  for j in range(len(index_triplet)-1): # iterate over the indices within each triplet
                      # Check if the index exists in kpts_conf

                      if index_triplet[j] >= len(kpts.conf[0]) or index_triplet[j+1] >= len(kpts.conf[0]):
                        continue
                      if kpts.conf[0][index_triplet[j]] < 0.5 or kpts.conf[0][index_triplet[j+1]] < 0.5:
                        continue

                      keypoint = kpts.xy[0, index_triplet[j]]
                      x, y = int(keypoint[0].item()), int(keypoint[1].item())
                      #t = cv2.circle(im0, (x, y), 5, colors[i], -1)

                      # Get the next keypoint
                      keypoint_next = kpts.xy[0, index_triplet[j+1]]
                      x_next, y_next = int(keypoint_next[0].item()), int(keypoint_next[1].item())

                      # Draw a dotted line between the current keypoint and the next one
                      draw_dotted_line(im0, (x, y), (x_next, y_next), colors[i], 2)

                      if j == len(index_triplet) - 2:
                          cv2.arrowedLine(im0, (x, y), (x_next, y_next), colors[i], 2)

              #https://donologue.tistory.com/398
              kpt8 = kpts.xy[0][8]
              kpt6 = kpts.xy[0][6]

              kpt5 = kpts.xy[0][5]
              kpt7 = kpts.xy[0][7]
              kpt9 = kpts.xy[0][9]

              distance1 = ((kpt7[0] - kpt5[0]) ** 2 + (kpt7[1] - kpt5[1]) ** 2) ** 0.5
              distance2 = ((kpt6[0] - kpt8[0]) ** 2 + (kpt6[1] - kpt8[1]) ** 2) ** 0.5

              radius1 = distance1 / 2.5
              radius2 = distance2 / 2.5

              cv2.circle(im0, (int(kpt7[0].item()), int(kpt7[1].item())), int(radius1), (255, 255, 255), 2)
              cv2.circle(im0, (int(kpt8[0].item()), int(kpt8[1].item())), int(radius2), (255, 255, 255), 2)

              overlay = im0.copy()
              cv2.circle(overlay, (int(kpt7[0].item()), int(kpt7[1].item())), int(radius1), (255, 255, 255), -1)
              cv2.circle(overlay, (int(kpt8[0].item()), int(kpt8[1].item())), int(radius2), (255, 255, 255), -1)
              alpha = 0.4
              im0 = cv2.addWeighted(overlay, alpha, im0, 1 - alpha, 0)

              percievedAngle = 360 - getAngle(kpt5,kpt7,kpt9)

              x_str = str(int(percievedAngle))
              print(x_str)
              #cv2.putText(im0, x_str, (int(kpt7[0].item()), int(kpt7[1].item())), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
              cv2.putText(im0, x_str, (int(kpt7[0].item()), int(kpt7[1].item())), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

        video_writer.write(im0)
    video_writer.release()
    cap.release()

cv2.destroyAllWindows()

In [47]:
print(kpts.xy)

tensor([], size=(1, 0, 2))


In [ ]:
for result in results:
    kptstest = result.keypoints.conf[0][15]
    print(kptstest)
    nk = kpts.shape[1]
    conf_index = []
    for i in enumerate(dotted_line_indices):
        conf_4 = kpts
        indices = kpts[0][15]



In [ ]:
print(kpts.xy[0][16])
print(kpts.xy)

# Instagram videos

In [ ]:
 for result in results:
    kpts_conf = result.keypoints.conf
    print(kpts_conf)


tensor([[0.9395, 0.9464, 0.7655, 0.8654, 0.2237, 0.9635, 0.8512, 0.9287, 0.6354, 0.9098, 0.6766, 0.6462, 0.4666, 0.1832, 0.0989, 0.0668, 0.0440]])


# Configure ther way to vizaualize

In [ ]:
import moviepy.editor
moviepy.editor.ipython_display("/content/output_folder/workouts_0.mp4",autoplay=1)
